In [35]:
from huggingface_hub import list_repo_files
import duckdb
import pandas as pd

# list all files
files = list_repo_files("tahoebio/Tahoe-100M", repo_type="dataset")

# build list of full HTTPS URLs for the shards
parquet_files = [
    f"https://huggingface.co/datasets/tahoebio/Tahoe-100M/resolve/main/{f}"
    for f in files
    if f.startswith("data/") and f.endswith(".parquet")
]
# Inspect schema of the first shard
duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet('{parquet_files[0]}')").show()

print("Total shards:", len(parquet_files))
print("First 3:", parquet_files[:3])

# filter lists
target_drugs = ["5-Fluorouracil", "5-azacytidine", "Oxaliplatin"]
overlap_lines = ["SNU-423", "NCI-H1792", "NCI-H460", "NCI-H2347",
                 "C32", "AsPC-1", "SK-MEL-2"]

drug_list = ",".join([f"'{d.lower()}'" for d in target_drugs])
line_list = ",".join([f"'{c.lower()}'" for c in overlap_lines])

# Look at distinct cell_line_id values from shard 0 (no drug filter)
df_lines = duckdb.sql(f"""
    SELECT DISTINCT cell_line_id
    FROM read_parquet('{parquet_files[0]}')
    LIMIT 50
""").df()

print(df_lines)


# run query across shards
df = duckdb.sql(f"""
    SELECT drug, cell_name
    FROM read_parquet({parquet_files})
    WHERE lower(drug) IN ({drug_list})
      AND lower(cell_name) IN ({line_list})
""").df()

print("Rows fetched:", len(df))
df.head()


┌────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name     │ column_type │  null   │   key   │ default │  extra  │
│      varchar       │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ genes              │ BIGINT[]    │ YES     │ NULL    │ NULL    │ NULL    │
│ expressions        │ FLOAT[]     │ YES     │ NULL    │ NULL    │ NULL    │
│ drug               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ sample             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ BARCODE_SUB_LIB_ID │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ cell_line_id       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ moa-fine           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ canonical_smiles   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ pubchem_cid        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │

BinderException: Binder Error: Referenced column "cell_name" not found in FROM clause!
Candidate bindings: "cell_line_id", "canonical_smiles", "genes", "pubchem_cid", "sample"

In [ ]:
# treated counts
treated_counts = (
    df.loc[~df["drug"].str.contains("dmso", case=False)]
      .groupby(["drug", "cell_name"])
      .size()
      .reset_index(name="n_treated")
)

# control counts
control_counts = (
    df.loc[df["drug"].str.contains("dmso", case=False)]
      .groupby("cell_name")
      .size()
      .reset_index(name="n_control")
)

# merge
candidate_df = pd.merge(treated_counts, control_counts, on="cell_name", how="left")
candidate_df["n_control"] = candidate_df["n_control"].fillna(0).astype(int)

candidate_df.to_csv("tahoe_candidate_pairs_with_controls.csv", index=False)
print("Saved: tahoe_candidate_pairs_with_controls.csv")
candidate_df.head()


Total files: 4427
.gitattributes
LICENSE.md
README.md
data/train-00000-of-03388.parquet
data/train-00001-of-03388.parquet
data/train-00002-of-03388.parquet
data/train-00003-of-03388.parquet
data/train-00004-of-03388.parquet
data/train-00005-of-03388.parquet
data/train-00006-of-03388.parquet
data/train-00007-of-03388.parquet
data/train-00008-of-03388.parquet
data/train-00009-of-03388.parquet
data/train-00010-of-03388.parquet
data/train-00011-of-03388.parquet
data/train-00012-of-03388.parquet
data/train-00013-of-03388.parquet
data/train-00014-of-03388.parquet
data/train-00015-of-03388.parquet
data/train-00016-of-03388.parquet
data/train-00017-of-03388.parquet
data/train-00018-of-03388.parquet
data/train-00019-of-03388.parquet
data/train-00020-of-03388.parquet
data/train-00021-of-03388.parquet
data/train-00022-of-03388.parquet
data/train-00023-of-03388.parquet
data/train-00024-of-03388.parquet
data/train-00025-of-03388.parquet
data/train-00026-of-03388.parquet
data/train-00027-of-03388.pa